# Recommender Code Along

The classic recommender tutorial uses the [movielens data set](https://grouplens.org/datasets/movielens/). It is similar to using the iris or MNIST data set for other algorithms. Let's do a code along to get an idea of how this all works!


Looking for more datasets? Check out: https://gist.github.com/entaroadun/1653794

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('rec').getOrCreate()

23/08/19 23:33:21 WARN Utils: Your hostname, Blade-15-Base-Model resolves to a loopback address: 127.0.1.1; using 192.168.0.22 instead (on interface wlo1)
23/08/19 23:33:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/19 23:33:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


With Collaborative filtering we make predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption is that if a user A has the same opinion as a user B on an issue, A is more likely to have B's opinion on a different issue x than to have the opinion on x of a user chosen randomly.

The image below (from Wikipedia) shows an example of collaborative filtering. At first, people rate different items (like videos, images, games). Then, the system makes predictions about a user's rating for an item not rated yet. The new predictions are built upon the existing ratings of other users with similar ratings with the active user. In the image, the system predicts that the user will not like the video.

<img src=https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif />

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.

Let's see this all in action!

In [3]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [4]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

In [5]:
data.head()

Row(movieId=2, rating=3.0, userId=0)

In [6]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [7]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



We can do a split to evaluate how well our model performed, but keep in mind that it is very hard to know conclusively how well a recommender system is truly working for some topics. Especially if subjectivity is involved, for example not everyone that loves star wars is going to love star trek, even though a recommendation system may suggest otherwise.

In [8]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [9]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating") # the last three parameters are the most essentials 
model = als.fit(training)

Now let's see hwo the model performed!

In [10]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [11]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|      0|   3.0|    28|   1.392265|
|      3|   1.0|    26|   1.095296|
|      4|   4.0|    26| -1.8181177|
|      2|   1.0|    12|-0.57001513|
|      0|   1.0|    22|  0.9272144|
|      6|   1.0|     1|  2.3954172|
|      3|   1.0|    13|  1.4398248|
|      4|   2.0|    13|  1.6252669|
|      5|   1.0|     6|-0.50500685|
|      9|   1.0|     6|  2.5179493|
|      7|   3.0|     3|-0.70338666|
|      6|   1.0|    20|   1.966013|
|      2|   1.0|    17|    1.60514|
|      4|   1.0|     9|  1.6426525|
|      6|   1.0|     9| 0.99787796|
|      6|   1.0|    17|  1.9812541|
|      1|   1.0|     4| -1.6596199|
|      6|   1.0|     4| 0.17095625|
|      9|   1.0|     4| -2.4235682|
|      1|   3.0|    25| -3.0382068|
+-------+------+------+-----------+
only showing top 20 rows



In [12]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 2.082945700041682


The RMSE described our error in terms of the stars rating column.

So now that we have the model, how would you actually supply a recommendation to a user?

The same way we did with the test data! For example:

In [13]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [14]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      9|    11|
|     12|    11|
|     16|    11|
|     19|    11|
|     30|    11|
|     36|    11|
|     37|    11|
|     40|    11|
|     47|    11|
|     51|    11|
|     61|    11|
|     67|    11|
|     69|    11|
|     70|    11|
|     71|    11|
|     82|    11|
|     86|    11|
|     94|    11|
|     97|    11|
+-------+------+



In [15]:
reccomendations = model.transform(single_user)

In [16]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     51|    11|  4.8759465|
|     69|    11|  3.5311306|
|     70|    11|  3.3047774|
|     67|    11|  3.1359107|
|     36|    11|  2.5572076|
|     16|    11|  1.4518279|
|     82|    11|  1.1602839|
|     30|    11|   1.061749|
|     97|    11| 0.55734336|
|     47|    11|  0.5312919|
|     61|    11| 0.25045395|
|     86|    11| 0.18645638|
|     19|    11|-0.37979588|
|      9|    11| -0.4299196|
|     94|    11| -0.6507803|
|     37|    11| -0.7949051|
|     12|    11| -1.1139294|
|     71|    11| -2.2246757|
|     40|    11| -2.3395464|
+-------+------+-----------+



# Great Job!

In [21]:
reccomendations["prediction"]

Column<'prediction'>

In [25]:
reccomendations.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- userId: integer (nullable = true)
 |-- prediction: float (nullable = false)



In [26]:
from pyspark.sql.functions import col

In [28]:
recommendations = reccomendations.withColumn("rounded_prediction", (col("prediction").cast("int")))


In [29]:
recommendations.show()

+-------+------+-----------+------------------+
|movieId|userId| prediction|rounded_prediction|
+-------+------+-----------+------------------+
|      9|    11| -0.4299196|                 0|
|     12|    11| -1.1139294|                -1|
|     16|    11|  1.4518279|                 1|
|     19|    11|-0.37979588|                 0|
|     30|    11|   1.061749|                 1|
|     36|    11|  2.5572076|                 2|
|     37|    11| -0.7949051|                 0|
|     40|    11| -2.3395464|                -2|
|     47|    11|  0.5312919|                 0|
|     51|    11|  4.8759465|                 4|
|     61|    11| 0.25045395|                 0|
|     67|    11|  3.1359107|                 3|
|     69|    11|  3.5311306|                 3|
|     70|    11|  3.3047774|                 3|
|     71|    11| -2.2246757|                -2|
|     82|    11|  1.1602839|                 1|
|     86|    11| 0.18645638|                 0|
|     94|    11| -0.6507803|            

In [33]:
# Find the minimum value in the "prediction" column
min_value = recommendations.selectExpr("min(prediction)").first()[0]

# Round the "prediction" column and create a new column with rounded integer values
recommendations = recommendations.withColumn("rounded_prediction", ((col("prediction")).cast("int") if (col("prediction")).cast("int") >= 0 else 0))

# Show the resulting DataFrame
recommendations.show()

ValueError: Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.

In [35]:
from pyspark.sql.functions import when

In [36]:
recommendations = recommendations.withColumn("rounded_prediction", when(col("prediction").cast("int") >= 0, col("prediction").cast("int")).otherwise(0))

In [38]:
recommendations.show()

+-------+------+-----------+------------------+
|movieId|userId| prediction|rounded_prediction|
+-------+------+-----------+------------------+
|      9|    11| -0.4299196|                 0|
|     12|    11| -1.1139294|                 0|
|     16|    11|  1.4518279|                 1|
|     19|    11|-0.37979588|                 0|
|     30|    11|   1.061749|                 1|
|     36|    11|  2.5572076|                 2|
|     37|    11| -0.7949051|                 0|
|     40|    11| -2.3395464|                 0|
|     47|    11|  0.5312919|                 0|
|     51|    11|  4.8759465|                 4|
|     61|    11| 0.25045395|                 0|
|     67|    11|  3.1359107|                 3|
|     69|    11|  3.5311306|                 3|
|     70|    11|  3.3047774|                 3|
|     71|    11| -2.2246757|                 0|
|     82|    11|  1.1602839|                 1|
|     86|    11| 0.18645638|                 0|
|     94|    11| -0.6507803|            

# MAGNIFIQUE !